# Google's Quick Draw Dataset and GANs
## Lawton Manning, John Farrell, and Liyao Zhang

*Before running this Notebook:* <br> Make sure to download the QuickDraw files corresponding to the '.npy' filetype.

More information on Google's Quick Draw dataset found in their repository here: https://github.com/googlecreativelab/quickdraw-dataset 

## Preliminary Data Prep:

In [16]:
import numpy as np
import pandas as pd

from os import listdir
from os.path import basename, isfile, join

from matplotlib.pyplot import imshow

In [2]:
metadata = pd.read_csv(join('data', 'metadata.csv'))

In [54]:
%reload_ext autoreload
%autoreload 2
from DataGenerator import DataGenerator

datagen = DataGenerator(metadata, 32)   # locally defined Datagen class specific to Google's QuickDraw
datagen.n_classes

345

## Creating the GAN Model:
In order to model our GAN, we must define the two sub-models that comprise GANs in general. Collectively, they will learn to create sketches from Google's QuickDraw 28x28 greyscale images.
    
1. Discriminator - given an input image, determine if the image is real or fake
2. Generator - learn how to generate an image which fools the discriminator (so that it cannot distinguish real/fake reliably)
    